本方法提供了两种运行思路，一种是原始数据处理，另一种使用我们已经处理好的进行训练
原始数据处理起来相对较慢，可以先尝试使用我们已经处理好的数据进行验证。


如果需要在B榜数据进行验证的话。需要在标题为 处理A榜数据哪里改成B榜数据的信息


## 库所需要的版本要求
sklearn->1.1.2
pandas->1.1.4
lightgbm->3.3.2 ->这个比较关键

## 导入所需要的库

In [1]:
import numpy as np
from glob import glob
import random
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import auc
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import random

# svm分类器
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
import sklearn.metrics
# 机器学习算法相关包
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
model_undersample = RandomUnderSampler(random_state=42)
from lightgbm import LGBMClassifier

## 读取数据部分

In [2]:
# 以下这部分是赛事提供的没做任何修改
data_path = 'Train'  # 存放数据的路径
pkl_files = glob(data_path + '/*.pkl')
len(pkl_files)

ind_pkl_files = []  # 存放标签为0的文件
ood_pkl_files = []  # 存放标签为1的文件
for each_path in tqdm(pkl_files):
    # 打开pkl文件
    pic = open(each_path, 'rb')
    this_pkl_file = pickle.load(pic)
    if this_pkl_file[1]['label'] == '00':
        ind_pkl_files.append(each_path)
    else:
        ood_pkl_files.append(each_path)

random.seed(0)
random.shuffle(ind_pkl_files)
random.shuffle(ood_pkl_files)


100%|██████████| 28389/28389 [00:00<00:00, 36117.06it/s]


## 下面这部分开始处理异常数据

In [3]:
data_df_norm = []  # 将结果保存为df
# 所需的特征 也是赛事提供的，这里需要所有的特征
columns = ['volt','current','soc','max_single_volt','min_single_volt','max_temp','min_temp','timestamp'] 
index = 1
for each_pkl in tqdm(ood_pkl_files):
    pic = open(each_pkl, 'rb')
    item = pickle.load(pic)
    data = item[0]
    temp_df = pd.DataFrame(data, columns=columns)
    data_label = item[1]['label']                # 打上标签
    data_mileage = float(item[1]['mileage'])     # 考虑mileage特征
    temp_df['pkl'] = each_pkl                    # 对应的文件名字
    temp_df['mileage'] = data_mileage 
    temp_df['label'] = data_label
    temp_df['custom_index'] = index              # 每条数据有一个索引 用来标识每条数据

    index += 1
    data_df_norm.append(temp_df)

data_df_norm = pd.concat(data_df_norm)

100%|██████████| 4654/4654 [00:05<00:00, 902.05it/s]


In [4]:
data_df_group = data_df_norm.groupby('custom_index')   # 按照每条数据对应的索引进行分组，方便后面对每条数据进行特征处理
group_data = []
for idx, data_ in tqdm(data_df_group):
    group_data.append(data_)

100%|██████████| 4654/4654 [00:00<00:00, 20198.37it/s]


In [6]:
def helper(x):
    '''
    x: 特征 
    比如 volt，会有序列的信息 比如第一时刻到256时刻的volt数据
    对这序列数据取到 最大值、最小值、均值、求和、median、方差
    这些特征用来反映这条时序数据的信息
    
    
    后续会对每一个特征都会进行这样的操作，另外还会引入新的特征序列，比如相对volt时序信息等等
    '''
    if isinstance(x, type(np.array([1,2,]))):
        x = pd.DataFrame(x)
    max_num = x.max()
    min_num = x.min()
    mean_num = x.mean()
    sum_num = x.sum()
    median_num = float(x.median())
    std_num = float(x.std())

    res = []
    for i in [max_num, min_num, mean_num, sum_num, median_num, std_num]:
        res.append(float(i))
    return res

In [8]:
def get_features(m, ):
    '''
    m: 表示每一条数据  序列长度*['volt','current','soc','max_single_volt','min_single_volt','max_temp','min_temp','timestamp', 'mileage']
    
    以下的思路是先对每一个已有的序列特征根据helper提取到他的一些标量化特征来表示这条序列特征的信息
    '''
    
    # 所有的特征信息
    features = []  
    # 遍历已有的特征，注意这里比赛事给的特征多了一个 mileage
    for i in columns:
        l = helper(m[i])  # 对每一个序列特征提取他的标量特征
        features += l     # 把它加到features
        
    # 以下是相对volt信息，也就是上一时刻的volt减去前一时刻的volt，构造的新的特征信息，这里做了一定的扩大用来放大特征
    # 这里基本上就是假设有1-256 时刻 就是。2-256的特征减去 1-255
    one_last_volt = np.array(m['volt'][1:])*10
    zero_last_volt = np.array(m['volt'][0:-1])*10
    new_volt = one_last_volt - zero_last_volt
    new_volt_rel = np.hstack([new_volt, np.array(m['volt'].iloc[-1]) - np.array(m['volt'].iloc[0])])
    # 相对信息的最大最小 mean/sum/median
    features+=helper(new_volt_rel)  # 对序列特征进行标量化处理
    
    # 同理这里是max_single_volt的相对特征，做了缩放
    one_last_volt = np.array(m['max_single_volt'][1:])*1000
    zero_last_volt = np.array(m['max_single_volt'][0:-1])*1000
    new_volt = one_last_volt - zero_last_volt
    new_max_single_volt = np.hstack([new_volt,np.array(m['max_single_volt'].iloc[-1]) - np.array(m['max_single_volt'].iloc[0])])
    features+=helper(new_max_single_volt)
    
    # 同理这里是min_single_volt的相对特征，做了缩放
    one_last_volt = np.array(m['min_single_volt'][1:])*1000
    zero_last_volt = np.array(m['min_single_volt'][0:-1])*1000
    new_volt = one_last_volt - zero_last_volt
    new_min_single_volt = np.hstack([new_volt,np.array(m['min_single_volt'].iloc[-1]) - np.array(m['min_single_volt'].iloc[0])])
    features+=helper(new_min_single_volt)
    
    # 这里对温度做了缩放信息
    new_max_temp_vari = m['max_temp'] - 200
    new_min_temp_vari = m['min_temp'] - 160
    features += helper(new_max_temp_vari)
    features += helper(new_min_temp_vari)
    
    # 这里是特征变化的信息 也就是最后一个时刻和第一个时刻的每一个特征的变化特征。
    last_one_diff = list(m[columns].iloc[-1] - m[columns].iloc[0])
    features += last_one_diff
    
    # 最大volt和最小volt的差特征
    max_min_volt_change = m['max_single_volt'] - m['min_single_volt']
    features += helper(max_min_volt_change)
    # 最大温度和最小温度的差特征
    max_min_temp_change = m['max_temp'] - m['min_temp']
    features += helper(max_min_temp_change)
    
    return features
    

## 以下是利用上面两个函数对异常时序数据进行标量化特征处理

核心想法描述：
由于给定的数据集正负样本极其不均衡，正样本数据极其稀少（相对负样本来说）。所以首先需要扩充数据来提升预测效果。
给定的数据是1-256条序列数据。为了扩充数据，需要对数据做切分，将一条数据变成多条数据，同时保证数据的时序性。
我们按照将一条数据划分成4条数据。

由于是时序数据，所以取间断时刻构成的数据也算时序数据，而且出现异常的状态肯定是一段时间的所以间断时刻不会改变它是异常的本质
比如现在有12条数据。我们按照每3个时刻为间断构建一条新的数据：
具体如下： \\ 
\\ \[1,2,3,4,5,6,7,8,9,10,11,12\]\\
\[0, 4, 8\]  \\
\[1, 5, 9\]  \\ 
\[2, 6, 10\] \\
\[3, 7, 11\] \\
这样一条时序数据被划分成了4条新的时序数据，
同样的1-256会被划分成4条新的数据

In [9]:
new_df = []   # 存储结果
columns = ['volt','current','soc','max_single_volt','min_single_volt','max_temp',
           'min_temp','timestamp', 'mileage',]   # 这里就是需要处理的序列特征多了一个 mileage
for idx, m in tqdm(enumerate(group_data)):  # 之前按照每条数据对应的索引进行了分组，现在遍历每一条数据
    for i in range(4):                      # 这里是我们核心的想法，参考上面的描述
        df = m.iloc[i::4]
        features = get_features(df)         # 特征处理
        temp_df = pd.DataFrame(features).T  # 将其保存
        temp_df['label'] = 1                # 打标签
        new_df.append(temp_df)

4654it [02:22, 32.66it/s]


### 以上就是异常数据的处理结果了存储到了new_df

In [28]:
new_df[0]

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,label
0,152.3,149.3,151.529687,9697.9,151.55,0.572014,-0.8,-24.2,-23.159375,-1482.2,...,0.41,0.006,0.000684,30.0,24.0,29.90625,1914.0,30.0,0.75,1


## 以下是正常数据的处理过程和异常处理一样

### 读取数据

In [10]:
data_df_norm = []
columns = ['volt','current','soc','max_single_volt','min_single_volt','max_temp','min_temp','timestamp', ]
index = 1
for each_pkl in tqdm(ind_pkl_files):
    pic = open(each_pkl, 'rb')
    item = pickle.load(pic)
    data = item[0]
    temp_df = pd.DataFrame(data, columns=columns)
    data_label = item[1]['label']
    data_mileage = float(item[1]['mileage'])
    temp_df['pkl'] = each_pkl
    temp_df['mileage'] = data_mileage
    temp_df['label'] = data_label
    temp_df['custom_index'] = index
    
    index += 1
    data_df_norm.append(temp_df)

data_df_norm = pd.concat(data_df_norm)

100%|██████████| 23735/23735 [00:25<00:00, 928.24it/s]


### 分组

In [11]:
data_df_group = data_df_norm.groupby('custom_index')
group_data = []
for idx, data_ in tqdm(data_df_group):
    group_data.append(data_)

100%|██████████| 23735/23735 [00:01<00:00, 15032.76it/s]


### 划分数据时序数据标量化

In [12]:
columns = ['volt','current','soc','max_single_volt','min_single_volt','max_temp','min_temp',
           'timestamp', 'mileage']
norm_feature_df = []
for idx, m in tqdm(enumerate(group_data)):
    for i in range(4):
        df = m.iloc[i::4]
        features = get_features(df)
        temp_df = pd.DataFrame(features).T
        temp_df['label'] = 0
        norm_feature_df.append(temp_df)

23735it [12:10, 32.49it/s]


## 以下是处理A榜数据

In [14]:
data_path = 'Test_A'  # 存放数据的路径
pkl_files = glob(data_path + '/*.pkl')
len(pkl_files)

6234

In [15]:
test_data = []
columns = ['volt','current','soc','max_single_volt','min_single_volt','max_temp','min_temp','timestamp', ]
index = 1
for each_pkl in tqdm(pkl_files):
    pic = open(each_pkl, 'rb')
    item = pickle.load(pic)
    data = item[0]
    temp_df = pd.DataFrame(data, columns=columns)
    data_mileage = float(item[1]['mileage'])
    temp_df['pkl'] = each_pkl
    temp_df['mileage'] = data_mileage
    temp_df['custom_index'] = index
    
    index += 1
    test_data.append(temp_df)

test_data = pd.concat(test_data)

100%|██████████| 6234/6234 [00:05<00:00, 1143.62it/s]


## 同样的数据会被划分成4组，所以需要数据分别存放，将最后的4组数据分别得到的预测结果取均值

In [16]:
columns = ['volt','current','soc','max_single_volt','min_single_volt','max_temp','min_temp','timestamp', 
           'mileage',]
#           'current_soc', 'max_temp_min_temp', 'volt_current', 'max_vol_max_temp', 'current_mileage']
data_df_group = test_data.groupby('custom_index')

group_data = []
test_df_odd = []
test_df_even = []
for idx, data_ in tqdm(data_df_group):
    group_data.append(data_)

test_df1 = []
test_df2 = []
test_df3 = []
test_df4 = []
for idx, m in tqdm(enumerate(group_data)):
    
    for i in range(4):
        
        df = m.iloc[i::4]
        features = get_features(df)
        temp_df = pd.DataFrame(features).T
        name = np.array(df['pkl'])
        temp_df['pkl'] = name[0]
#         temp_df['label'] = 0
        if i==0:
            test_df1.append(temp_df)
        elif i==1:
            test_df2.append(temp_df)
        elif i==2:
            test_df3.append(temp_df)
        else:
            test_df4.append(temp_df)

100%|██████████| 6234/6234 [00:00<00:00, 24745.25it/s]
6234it [03:11, 32.55it/s]


In [17]:

# 4组数据分别存放
test_df1 = pd.concat(test_df1)
test_df2 = pd.concat(test_df2)
test_df3 = pd.concat(test_df3)
test_df4 = pd.concat(test_df4)
# 存放到一个list
test_df = [test_df1, test_df2, test_df3, test_df4]

### 整个上述部分就是原始数据处理的部分了，下面是直接读取我们已经处理过的训练数据

In [39]:
import pickle
with open("lgb_feature_10-1-90.pkl", 'rb') as f:
    data_new = pickle.load(f)

In [43]:
# 可以判断 原始数据处理后的数据是否和我们提供的已处理数据是否一致 data在下面生成。
(data_new != data).sum().sum()

0

## 以下是训练训练部分

In [18]:
# 将正常和异常合并
data = new_df + norm_feature_df
data = pd.concat(data)

In [20]:
data

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,label
0,152.3,149.3,151.529687,9697.9,151.55,0.572014,-0.8,-24.2,-23.159375,-1482.2,...,0.410,0.006,0.000684,30.0,24.0,29.90625,1914.0,30.0,0.750000,1
0,152.3,149.5,151.546875,9699.0,151.55,0.540420,-3.0,-24.2,-23.228125,-1486.6,...,0.406,0.006,0.000648,30.0,30.0,30.00000,1920.0,30.0,0.000000,1
0,152.3,149.5,151.560937,9699.9,151.55,0.517202,-4.5,-24.2,-23.443750,-1500.4,...,0.412,0.006,0.000687,30.0,24.0,29.90625,1914.0,30.0,0.750000,1
0,152.3,149.5,151.562500,9700.0,151.55,0.515321,-4.5,-24.2,-23.453125,-1501.0,...,0.407,0.006,0.000698,36.0,24.0,30.09375,1926.0,30.0,1.305893,1
0,161.7,158.3,159.853125,10230.6,159.60,0.943140,-46.0,-46.0,-46.000000,-2944.0,...,0.708,0.011,0.000432,24.0,18.0,19.12500,1224.0,18.0,2.360387,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,158.2,157.6,157.889063,10104.9,157.80,0.221909,-6.0,-6.1,-6.078125,-389.0,...,0.683,0.011,0.000592,6.0,6.0,6.00000,384.0,6.0,0.000000,0
0,153.0,152.8,152.903125,9785.8,152.90,0.077600,-4.3,-4.4,-4.398438,-281.5,...,0.199,0.003,0.000567,12.0,12.0,12.00000,768.0,12.0,0.000000,0
0,153.0,152.8,152.904687,9785.9,152.90,0.078538,-4.4,-4.4,-4.400000,-281.6,...,0.200,0.003,0.000604,12.0,12.0,12.00000,768.0,12.0,0.000000,0
0,153.0,152.8,152.903125,9785.8,152.90,0.077600,-4.4,-4.4,-4.400000,-281.6,...,0.203,0.003,0.000579,12.0,12.0,12.00000,768.0,12.0,0.000000,0


In [19]:
# 总共会构建出来105个特征，需要对其进行筛选，
features_index = [int(i) for i in ['1',  
                                   '6', '8','11','10', 
                                   '12', '16', '15', '17', 
                                   '23', 
                                   '30', '34','32', 
                                   '36', '41', '40', 
                                   '47',   
                                   '48', '51' ,'52', '53',  
                                   '55', '54',   
                                   '61',   
                                   '72', '73', '74', 
                                   '79', '80', '78', 
                                   '89', '91', 
                                   '100', '103']] 

In [44]:
# 标签数据 划分
target = 'label'
x = data.iloc[:, features_index]
y = LabelEncoder().fit_transform(data[target])

In [45]:
# 使用欠采样方法
print('欠采样前', len(x), len(y))
x, y = model_undersample.fit_resample(x, y)
print('欠采样后', len(x), len(y))

欠采样前 113556 113556
欠采样后 37232 37232


In [46]:
# 划分训练测试数据
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20, random_state=42, shuffle=True)

In [47]:
# 基础模型选择为LGBMClassifier
model5 = LGBMClassifier(boosting_type='rf', learning_rate=0.0001,max_depth=6,num_leaves=2**6,
                       reg_lambda=0.02,random_state=2, is_unbalance=False,
                       bagging_freq=10,
                       bagging_fraction= 0.5,
                       feature_fraction=0.5,
                       reg_alpha=0.01)
model5.fit(x, y)
pred5_score = model5.predict_proba(test_x)[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(test_y, pred5_score, pos_label=1)
print('model5：\n', model5, '\nauc:', metrics.auc(fpr, tpr))

# 正确的预测结果auc: 0.9987597833542866，如果不对可能导致A榜结果预测不准确，
# 处理办法可以参考我们提供已经处理好的数据，或者将LGBMClassifier包升级 满足版本要求


[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
model5：
 LGBMClassifier(bagging_fraction=0.5, bagging_freq=10, boosting_type='rf',
               feature_fraction=0.5, is_unbalance=False, learning_rate=0.0001,
               max_depth=6, num_leaves=64, random_state=2, reg_alpha=0.01,
               reg_lambda=0.02) 
auc: 0.9987597833542866


In [35]:
# 计算A榜的预测结果，将4组预测结果取平均值
def compute_test():
    score = 0
    for i in test_df:
        score += model5.predict_proba(i.iloc[:, features_index])[:, 1]
    
    score /= 4
    submision_df = pd.DataFrame()
    submision_df['file_name'] = test_df1['pkl'].apply(lambda x:x[7:])
    submision_df['score'] = score
    submision_df.to_csv('submision.csv', index=False)
    return submision_df

In [36]:
submision_df = compute_test()

In [38]:
submision_df[:10]

,file_name,score
0,7096.pkl,0.154747
0,6421.pkl,0.179055
0,10799.pkl,0.122404
0,3655.pkl,0.122300
0,10947.pkl,0.121699
0,4739.pkl,0.137857
0,8268.pkl,0.125588
0,7863.pkl,0.120588
0,6809.pkl,0.127282
0,8242.pkl,0.119230
